In [ ]:
!pip install gluonts
!pip install --upgrade datasets
!pip install utilsforecast

In [ ]:
!git clone https://github.com/GiuliaGhisolfi/TSFM-ZeroShotEval
%cd TSFM-ZeroShotEval/src

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

### Import data

In [ ]:
DATASET_NAME = ["exchange_rate", "ercot", "dominick"]

### Results file

In [ ]:
import os
from utils.save_results import create_output_file, save_results

output_file_name = "chronos_benchmark_results.csv"

output_dir="results"
csv_file_path = os.path.join(output_dir, output_file_name)

create_output_file(output_file_name, output_dir)

## Evaluation

In [ ]:
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
from utils.load_chronos_data import load_data

In [11]:
from utils.metrics import get_metrics

# Instantiate the metrics
metrics = get_metrics()

### Chronos

In [ ]:
CHRONOS_MODEL_NAME = ["chronos_bolt_tiny", "chronos_bolt_mini", "chronos_bolt_small", "chronos_bolt_base"]

CHRONOS_MODEL = ["amazon/chronos-bolt-tiny", "amazon/chronos-bolt-mini",
    "amazon/chronos-bolt-small", "amazon/chronos-bolt-base"]
# "amazon/chronos-t5-tiny", "amazon/chronos-t5-mini", "amazon/chronos-t5-small",
# "amazon/chronos-t5-base", "amazon/chronos-t5-large",

In [ ]:
from utils.chronos_predictor import ChronosPredictor

for model_name, model_path in zip(CHRONOS_MODEL_NAME, CHRONOS_MODEL):
    for ds_name in DATASET_NAME:
        print(f"Processing dataset: {ds_name}")

        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset, prediction_length, frequency, domain, num_variates = load_data(ds_name, term)
            season_length = get_seasonality(frequency)

            predictor = ChronosPredictor(
                model_path=model_path,
                num_samples=20,
                prediction_length=prediction_length,
                # Change device_map to "cpu" to run on CPU or "cuda" to run on GPU
                device_map="cuda",
            )

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, model_name, domain, num_variates, ds_name, csv_file_path)

### Moirai

In [ ]:
MOIRAI_MODEL_NAME = ["moirai_small", "moirai_base", "moirai_large"]

MOIRAI_MODEL = ["Salesforce/moirai-1.1-R-small", "Salesforce/moirai-1.1-R-base", 
    "Salesforce/moirai-1.1-R-large"]
#"Salesforce/moirai-moe-1.0-R-base", "Salesforce/moirai-moe-1.0-R-small"

In [ ]:
from utils.moirai_predictor import load_predictor

for model_name, model_path in zip(MOIRAI_MODEL_NAME, MOIRAI_MODEL):
    for ds_name in DATASET_NAME:
        print(f"Processing dataset: {ds_name}")

        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset, prediction_length, frequency, domain, num_variates = load_data(ds_name, term)
            season_length = get_seasonality(frequency)

            predictor = load_predictor(model_path, prediction_length, num_variates)

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, model_name, domain, num_variates, ds_name, csv_file_path)

### TimesFM

In [ ]:
TIMESFM_MODEL_NAME = ["timesfm2", "timesfm1"]

TIMESFM_MODEL = ["google/timesfm-2.0-500m-pytorch", "google/timesfm-1.0-200m-pytorch"]

In [ ]:
from utils.timesfm_predictor import load_predictor

for model_name, model_path in zip(TIMESFM_MODEL_NAME, TIMESFM_MODEL):
    for ds_name in DATASET_NAME:
        print(f"Processing dataset: {ds_name}")

        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset, prediction_length, frequency, domain, num_variates = load_data(ds_name, term)
            season_length = get_seasonality(frequency)

            predictor = load_predictor(model_path, prediction_length, frequency)

            res = evaluate_model(
                predictor,
                test_data=dataset,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            save_results(res, ds_config, model_name, domain, num_variates, ds_name, csv_file_path)